In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Huggingface.utils import HuggingfaceDatasetManager, explain_text, TransformersModelTypeEnum, ModelManager

This project is written and tested in Python 3.7.13


## Load model from Huggingface

In [3]:
model_type = TransformersModelTypeEnum.GA_DISTIL_ROBERTA_BASE_FINETUNED_FAKE_NEWS
model_manager = ModelManager(model_type)

Using device: cuda


## Load the respective dataset for the model

In [4]:
dataset_manager = HuggingfaceDatasetManager(model_type)

Loading instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/True.csv
Loaded 21417 instances from /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/True.csv
Loading instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/Fake.csv
Loaded 23481 instances from /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/Fake.csv


In [5]:
true_samples = dataset_manager.fetch_true_samples(sample_count=10)
true_sample_shap_values = explain_text(model_manager, text=true_samples[0])

Getting the following indexes: [ 4097 16469  4867  8834 10208  7209 12863  4676 13252 21015]
Explaining the following text: 
<--------------------------------------------------------------------------------------------->
CHICAGO (Reuters) - Barack Obama, making his first major appearance since leaving the White House, made no mention on Monday of his successor, Donald Trump, but urged young people to get more involved in their communities at a time of stark political divides.     “What’s been going on since I’ve been gone?” joked the former Democratic president as he moderated an event at the University of Chicago in the city where he began his political career and which will be the site of his presidential library. Obama, who once taught constitutional law at the school, recalled starting out as a young community organizer in the city and told a panel of six current and former students that he decided to focus his post-presidency on encouraging young people to engage with their commun

Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors


[{'label': 'LABEL_0', 'score': 5.944190706941299e-05}, {'label': 'LABEL_1', 'score': 0.9999405145645142}]
Predicted Fake with score: 5.944190706941299e-05
Predicted True with score: 0.9999405145645142


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
